In [20]:
# Parameters
import numpy as np

rho_a = 1600 #kg/m^3
# active and frozen soil probably have similar densities, with frozen having more water content
# 1.6 - 0.2 g/cm3. error on higher side because
rho_p = 2000
rho_w = 1000
K_a = 1e-3 
# frozen soil has very low permeability, 10^-6 m/s
K_p = 1e-6 
g = 9.81
mu = 1e-3


In [24]:
# Discretization
layers = 10
dy = 1.0  # Distance between layers in meters
dt = 1.0  # Time step in seconds
total_time = 10000000  # Total simulation time in seconds
A = 1.0 * dy #assume width of 1 meter 

# Geometry 
al_wl_boundary = int(3)
wl_pl_boundary = int(7)

# Initialize depth, pressure, and permeability arrays
depths = np.arange(0, layers * dy, dy)  
ov_pressures = np.zeros(layers)
pw_pressures = np.zeros(layers)
K = np.zeros(layers)

In [25]:
# Generate Initial Pressure Distribution
atm_pressure = 101325  # atmospheric pressure in Pa, not needed?

for i in range(layers):
    # Active Layer
    if i <= al_wl_boundary:
        ov_pressures[i] = rho_a * g * depths[i]
        K[i] = K_a
    # Water Layer
    elif i >= al_wl_boundary and i<= wl_pl_boundary :
        ov_pressures[i] = rho_w *g * (depths[i] - depths[al_wl_boundary]) + (rho_a * g * depths[al_wl_boundary]) 
        #Assuming a very high hydraulic conductivity 
        K[i] = 1e-1
    # Permafrost Layer 
    else:
        ov_pressures[i] = rho_p *g * (depths[i] - depths[wl_pl_boundary]) + (rho_w * g * depths[wl_pl_boundary]) + (rho_a * g * depths[al_wl_boundary])
        K[i] = K_p

# Initalize Pore water pressures 
for i in range(layers):
    # Active layer, assuming moist but not saturated soil. Negative pressure
    if i <= al_wl_boundary:
        pw_pressures[i] = -100
    # Pore water pressure at water level is 0 
    elif i >= al_wl_boundary and i<= wl_pl_boundary :
        pw_pressures[i] = rho_w*g * (i - al_wl_boundary)
    # Pore water pressure below water level
    else:
        #pw_pressures[i] = rho_w*g*(i - wl_pl_boundary)
        pw_pressures[i] = -100
    



In [26]:
# Solver Loop: Darcy Flow and Update Pressure
for t in np.arange(0, total_time, dt):
    # Calculate flow rates between layers based on Darcy's Law, depends on pore pressure grad
    Q = np.zeros(layers - 1)
    for i in range(1, layers):
        pore_grad = (pw_pressures[i] - pw_pressures[i-1]) / dy
        Q[i-1] = -K[i] * A * pore_grad / (rho_w *g)
    # Update pressures based on flow rates
    for i in range(1, layers-1):
        pw_pressures[i] += Q[i-1] * dt / A - Q[i] * dt / A
        # Check if pore water pressure exceeds lithostatic pressure
        if pw_pressures[i] >= ov_pressures[i]:
            print(f"Simulation stopped early at time {t}s. Pore water pressure exceeds lithostatic pressure at layer {i}.")
            break

    # Break out of the outer loop if the condition is met in any layer
    if any(ov_pressures >= pw_pressures):
        break


In [14]:
print(pw_pressures)

[-1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02 -1.00000000e+02
 -9.98989806e+01  9.80999898e+03  1.96200000e+04  2.94300000e+04
  3.92400000e+04  4.90500000e+04  5.88600000e+04  6.86700000e+04
  7.84800000e+04  8.82900000e+04  9.81000000e+04  1.07910000e+05
  1.17720000e+05  1.27530000e+05  1.37340000e+05  1.47150000e+05
  1.56960000e+05  1.66770000e+05  1.76580000e+05  1.86390000e+05
  1.96200000e+05  2.06010000e+05  2.15820000e+05  2.25630000e+05
  2.35440000e+05  2.45250000e+05  2.55060000e+05  2.64870000e+05
  2.74680000e+05  2.84490000e+05  2.94300000e+05  3.04110000e+05
  3.13920000e+05  3.23730000e+05  3.33540000e+05  3.43350000e+05
  3.53160000e+05  3.62970000e+05  3.72780000e+05  3.82590000e+05
  3.92400000e+05  4.02210

In [15]:
print(ov_pressures)

[      0.   15696.   31392.   47088.   62784.   78480.   94176.  109872.
  125568.  141264.  156960.  172656.  188352.  204048.  219744.  235440.
  251136.  266832.  282528.  298224.  313920.  323730.  333540.  343350.
  353160.  362970.  372780.  382590.  392400.  402210.  412020.  421830.
  431640.  441450.  451260.  461070.  470880.  480690.  490500.  500310.
  510120.  519930.  529740.  539550.  549360.  559170.  568980.  578790.
  588600.  598410.  608220.  618030.  627840.  637650.  647460.  657270.
  667080.  676890.  686700.  696510.  706320.  716130.  725940.  735750.
  745560.  755370.  971190.  990810. 1010430. 1030050.]
